# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

In [1]:
## importing all the packages necessary

import pandas as pd
import csv
import json
import requests
import requests
from bs4 import BeautifulSoup

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [2]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temp, alertness, inspired_oxygen, performance_status):
    """(str, int, int, float, int, int, float, str, str, str) -> (float)
    This function takes the sex, age, respiratory rate, oxygen saturation, heart rate, systolic bp, temperature,alertness, inspired_oxygen, 
    performance status parameters and returns the severity percentile.
    
    >>> priest('FEMALE',66,19,0.96,135,115,38.1,'ALERT','AIR','bed/chair bound, no self-care')
    0.34
    
    >>> priest('not known', 40, 5,0.96, 135, 105, 37, 'ALERT', 'air', 'limited self-care' )
    ValueError: Gender is not recognizable. You gave me a 'not known'
    
    >>> priest('male', 20, 13, 0.91, 120, 100, 36.0, "AlerT", 'aIr', "limited aCtivity, can self-care")
    0.34
    """
    
    ## Converting the string variables to lowercase
    
    sex = sex.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()
    severity_score = 0
    severity_percentile = 0
    
    ## writing coinditions for SEX and adding to SEVERITY SCORE 
    
    if  sex == "male":
        severity_score += 1
    elif sex == "female":
        severity_score += 0
    else:
        raise ValueError ('Gender is not recognizable. You gave me a {}'.format(print(sex)))
    
    ## writing coinditions for AGE and adding to SEVERITY SCORE 
    
    if age < 0:
        raise ValueError('Age is not Valid. You gave me a {}'.format(print(age)))
    elif age >= 16 and age <= 49:
        severity_score += 0
    elif age >= 50 and age <= 65:
        severity_score += 2
    elif age >= 66 and age <= 80:
        severity_score += 3
    else:
        severity_score += 4

    ## writing coinditions for RESPIRATORY RATE and adding to SEVERITY SCORE 
 
    if respiratory_rate < 0:
        raise ValueError('Error in Respiratory Rate. You gave me a {}'.format(print(respiratory_rate)))
    elif respiratory_rate > 0 and respiratory_rate < 9: 
        severity_score += 3
    elif respiratory_rate >= 9 and respiratory_rate <= 11:
        severity_score += 1
    elif respiratory_rate >= 12 and respiratory_rate <= 20:
        severity_score += 0
    elif respiratory_rate >= 21 and respiratory_rate <= 24:
        severity_score += 2
    else:
        severity_score += 3

    ## writing coinditions for OXYGEN SATURATION and adding to SEVERITY SCORE 

    if oxygen_saturation <0:
        raise ValueError('Invalid percentage of oxygen saturation. You gave me a {}'.format(print(oxygen_saturation)))
    elif oxygen_saturation < 0.92:
        severity_score += 3
    elif oxygen_saturation >= 0.92 and oxygen_saturation <= 0.93:
        severity_score += 2   
    elif oxygen_saturation >= 0.94 and oxygen_saturation <= 0.95:
        severity_score += 1
   
    else:
        severity_score += 0
      
    ## writing coinditions for HEART RATE and adding to SEVERITY SCORE 

    if heart_rate <0:
        raise ValueError('Heart rate is invalid. You gave me a {}'.format(print(heart_rate)))
    elif heart_rate < 41 or heart_rate > 130:
        severity_score += 3
    elif heart_rate >= 111 and heart_rate <= 130:
        severity_score += 2
    elif (heart_rate >= 41 and heart_rate <= 50) or (heart_rate >= 91 and heart_rate <= 110):
        severity_score += 1
    else:
        severity_score += 0

    ## writing coinditions for SYSTOLIC BP and adding to SEVERITY SCORE 
    
    if systolic_bp < 0:
        raise ValueError('Enter correct BP value. You gave me a {}'.format(print(systolic_bp)))
    elif systolic_bp < 91 or systolic_bp > 219:
        severity_score += 3
    elif systolic_bp >= 91 and systolic_bp <= 100:
        severity_score += 2
    elif systolic_bp >= 101 and systolic_bp <= 110:
        severity_score += 1
    else:
        severity_score += 0
    
    ## writing coinditions for TEMPERATURE and adding to SEVERITY SCORE 
    
    if temp < 35.1:
        severity_score += 3
    elif temp >= 35.1 and temp <= 36.0 or temp >= 38.1 and temp <= 39.0:
        severity_score += 1
    elif temp >= 36.1 and temp <= 38.0:
        severity_score += 0
    else:
        severity_score += 2
   
    ## writing coinditions for ALERTNESS and adding to SEVERITY SCORE  
    
    if alertness == "confused or not alert":
        severity_score += 3
    elif alertness == "alert":
        severity_score += 0
    else:
        raise ValueError ('Alertness is not recognizable. You gave me a {}'.format(print(alertness)))

    ## writing coinditions for INSPIRED OXYGEN and adding to SEVERITY SCORE 
    
    if inspired_oxygen == "supplemental oxygen":
        severity_score += 2
    elif inspired_oxygen == "air":
        severity_score += 0
    else:
        raise ValueError ('Inspired Oxygen is not recognizable. You gave me a {}'.format(print(inspired_oxygen)))    
        
    ## writing coinditions for PERFORMANCE STATUS and adding to SEVERITY SCORE 
    
    if performance_status == "bed/chair bound, no self-care":
        severity_score += 4
    elif performance_status == "limited strenuous activity, can do light activity":
        severity_score += 1
    elif performance_status == "limited activity, can self-care":
        severity_score += 2
    elif performance_status == "limited self-care":
        severity_score += 3
    elif performance_status == "unrestricted normal activity":
        severity_score += 0
    else:
        raise ValueError('Performance Status is not recognizable. You gave me a {}'.format(print(performance_status)))

    ## calculating SEVERITY PERCENTILE from SEVERITY SCORE

    if severity_score >= 0 and severity_score <= 1:
        severity_percentile += 0.01
    elif severity_score >= 2 and severity_score <= 3:
        severity_percentile += 0.02
    elif severity_score == 4:
        severity_percentile += 0.03
    elif severity_score == 5:
        severity_percentile += 0.09
    elif severity_score == 6:
        severity_percentile += 0.15
    elif severity_score == 7:
        severity_percentile += 0.18
    elif severity_score == 8:
        severity_percentile += 0.22
    elif severity_score == 9:
        severity_percentile += 0.26
    elif severity_score == 10:
        severity_percentile += 0.29
    elif severity_score == 11:
        severity_percentile += 0.34
    elif severity_score == 12:
        severity_percentile += 0.38
    elif severity_score == 13:
        severity_percentile += 0.46
    elif severity_score == 14:
        severity_percentile += 0.47
    elif severity_score == 15:
        severity_percentile += 0.49
    elif severity_score == 16:
        severity_percentile += 0.55
    elif severity_score >= 17 and severity_score <= 25:
        severity_percentile += 0.59
    else:
        severity_percentile += 0.99

    ## returining SEVERITY PERCENTILE    
        
    return severity_percentile

In [3]:
import doctest
doctest.run_docstring_examples(priest, globals(), verbose=True)

Finding tests in NoName
Trying:
    priest('FEMALE',66,19,0.96,135,115,38.1,'ALERT','AIR','bed/chair bound, no self-care')
Expecting:
    0.34
ok
Trying:
    priest('not known', 40, 5,0.96, 135, 105, 37, 'ALERT', 'air', 'limited self-care' )
Expecting:
    ValueError: Gender is not recognizable. You gave me a 'not known'
**********************************************************************
File "__main__", line 9, in NoName
Failed example:
    priest('not known', 40, 5,0.96, 135, 105, 37, 'ALERT', 'air', 'limited self-care' )
Exception raised:
    Traceback (most recent call last):
      File "/opt/tljh/user/lib/python3.6/doctest.py", line 1330, in __run
        compileflags, 1), test.globs)
      File "<doctest NoName[1]>", line 1, in <module>
        priest('not known', 40, 5,0.96, 135, 105, 37, 'ALERT', 'air', 'limited self-care' )
      File "<ipython-input-2-321aefab8d4b>", line 32, in priest
        raise ValueError ('Gender is not recognizable. You gave me a {}'.format(print(se

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [4]:
def find_hospital(age,sex, risk):
    """(int, str, float) -> (str)
    This function takes the age, sex, and risk parameters and returns the hospital name as a string.
    
    >>> find_hospital(40, 'male', 0.1)
    'Southwest Hospital and Medical Center'
    
    >>> find_hospital(20, 'female', 0.96)
    'Emory Dunwoody Medical Center'
    
    >>> find_hospital(72, 'male', 0.22)
    'Wesley Woods Geriatric Hospital'
    
    >>> find_hospital(60, 'male', 0.09)
    'Southwest Hospital and Medical Center'
    
    >>> find_hospital(10, 'female', 0.15)
    'Childrens Healthcare of Atlanta at Scottish Rite'
    
    """
    
    url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
    response = requests.get(url)
    hospital = response.json()['hospital']
    hospital_name = hospital
    if hospital_name == response.json()['hospital']:
        return hospital_name
    else:
        return None

In [5]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(), verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(20, 'female', 0.96)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(72, 'male', 0.22)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(60, 'male', 0.09)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(10, 'female', 0.15)
Expecting:
    'Childrens Healthcare of Atlanta at Scottish Rite'
ok


## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [6]:
def get_address(hospital):
    """(str) -> (str)
    This function takes hospital as an input and returns the address of the hospital.
    
    >>> get_address('SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA')
    '1821 CLIFTON ROAD NE'
    
    >>> get_address('Wesley Woods Geriatric Hospital')
    '1821 CLIFTON ROAD, NE'
    
    >>> get_address('SHEPHERD CENTER')
    '2020 PEACHTREE ROAD NW'
    
    """
    
    ## getting the information in the HTML
    
    url = 'https://www.officialusa.com/stateguides/health/hospitals/georgia.html'
    response = requests.get(url)
    hospital_address = BeautifulSoup(response.text)
    
    ## appending the headers of the table in the url to the empty list called HEADERS 
    
    headers = []
    hospital = hospital.title()
    for cell in hospital_address.find_all('th'):
        headers.append(cell.text)

    ## assigning a variable name to empty lists
    
    address_list = []
    hospital_details_list = []

    ## looping through all the states to find a particular state (ATLANTA)
    
    for states in hospital_address.find_all('tr'):
        hospital_details = []
        hospital_list = []
        for city in states.find_all('td'):
             if city.text == 'ATLANTA':

    ## Finding all the details of the hospitals in ATLANTA and appending them to list of lists
                    
                for details in city.find_next_siblings('td'):
                    detail = details.text
                    hospital_details.append(detail)
                    if detail[0] == hospital:
                        hospital_list.append(detail[0])
                
     ## checking if the hospital is present in the hospital list or not
    
                for name in hospital_list:
                    if name != hospital:
                        raise TypeError('The hospital is not present')
        
        
        ## finding if there are duplicates in the hospital list, if not appending all the lists to a bigger list 
        
                if len(hospital_list)>1:
                    raise TypeError('Duplicates Found')
                else:   
                    hospital_details_list.append(hospital_details)
                    
            

    ## looping through the above created list to convert each value to a header in the new dictionary called place
                
    for value in hospital_details_list:
        place = {
            headers[1]:value[0],
            headers[2]:value[1],
            headers[3]:value[2],
            headers[4]:value[3],
            headers[5]:value[4],
            headers[6]:value[5]
        }
        
    ## appending the dictionary to address_list 
        
        address_list.append(place)
    
    ## looping through the address_list to find the hospital
    
    for address in address_list:
        hospital_name = address['Hospital Name'].title()
        hospital_address = address['ADDRESS']
        if hospital_name == hospital:
            return hospital_address
            

In [7]:
import doctest
doctest.run_docstring_examples(get_address, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_address('SELECT SPECIALTY HOSPITAL - NORTHEAST ATLANTA')
Expecting:
    '1821 CLIFTON ROAD NE'
ok
Trying:
    get_address('Wesley Woods Geriatric Hospital')
Expecting:
    '1821 CLIFTON ROAD, NE'
ok
Trying:
    get_address('SHEPHERD CENTER')
Expecting:
    '2020 PEACHTREE ROAD NW'
ok


## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [8]:
def process_people(file_name):
    """(path) -> (list)
    This function takes the path of the file and returns the list of dictionary containing patient_number as keys and the list of the 
    parameters namely sex, age, breath, oxygen saturation, heart rate, systolic bp, temperature, inspired_oxygen, performance status, risk,
    hospital, and address of the hospital.
    
    """
    
    ## declaring needer empty lists
    
    patient_number = []
    risk_list = []
    hospital_name_list = []
    hospital_address_list = []
    patient_data = []
    final_list = []
    
    ## reading the file
    
    file_name = '/data/people.psv'
    dataframe = pd.read_csv(file_name,sep='|')

    ## looping through dataframe and extracting the patient number and appending it to the patient_number list
    
    for x in dataframe['patient']:
        patient_number.append(x)

    ## looping through the dataframe and calculating the severity percentile. Appending it to the the risk_list
        
    for i in range(len(dataframe)):
        risk = priest(dataframe.loc[i, "sex"], dataframe.loc[i, "age"],dataframe.loc[i, 'breath'],dataframe.loc[i, 'o2 sat'],
                      dataframe.loc[i,'heart'],dataframe.loc[i, 'systolic bp'],dataframe.loc[i, 'temp'],dataframe.loc[i, 'alertness'],
                      dataframe.loc[i, 'inspired'],dataframe.loc[i, 'status'])
        risk_list.append(risk)
        
    ## creating a column of risk scores and concatenating it to the new data frame called data_with_risk

    risk_column = pd.DataFrame({"risk":risk_list}, columns = ['risk'])
    data_with_risk = pd.concat([dataframe, risk_column], axis = 1)
    
    ## looping through the data_with_risk and finding the hospital. Appending it to the the hospital_name_list

    for i in range(len(data_with_risk)):
        hospital_name = find_hospital(data_with_risk.loc[i, "age"], data_with_risk.loc[i, "sex"],data_with_risk.loc[i,"risk"])
        hospital_name_list.append(hospital_name)
        
    ## creating a column of hospital_name and concatenating it to the new data frame called data_with_hospital
        
    hospital_name_column = pd.DataFrame({"hospital_name":hospital_name_list}, columns = ['hospital_name'])
    data_with_hospital = pd.concat([data_with_risk, hospital_name_column], axis = 1)

    ## looping through the data_with_hospital and finding the hospital address. Appending it to the the hospital_address_list
    
    for i in range(len(data_with_hospital)):
        hospital_address = get_address(data_with_hospital.loc[i, "hospital_name"])
        hospital_address_list.append(hospital_address)
        
    ## creating a column of hospital_address and concatenating it to the new data frame called data_with_address
        
    hospital_address_column = pd.DataFrame({"hospital_address":hospital_address_list}, columns = ['hospital_address'])
    data_with_address = pd.concat([data_with_hospital, hospital_address_column], axis = 1)

    ## looping through the data_with_address dataframe and appending all the values to a list called patient_data
    
    for i in range(len(data_with_address)):
        all_columns = [data_with_address.loc[i, "sex"], data_with_address.loc[i, "age"],data_with_address.loc[i, 'breath'],data_with_address.loc[i, 'o2 sat'],
               data_with_address.loc[i,'heart'],data_with_address.loc[i, 'systolic bp'],data_with_address.loc[i, 'temp'],data_with_address.loc[i, 'alertness'],
               data_with_address.loc[i, 'inspired'],data_with_address.loc[i, 'status'], data_with_address.loc[i, 'risk'],data_with_address.loc[i, 'hospital_name'],
               data_with_address.loc[i,'hospital_address']]
    
        patient_data.append(all_columns)
        
    ## creating a dictionary with patient_numbers as keys and patient_data as values. AApending them to a final_list 

    dictionary = dict(zip(patient_number, patient_data))
    final_list.append(dictionary)
    
    ## returning final_list
    
    return final_list

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

In [9]:
## loading the json file as check 

with open('/data/people_results.json') as check:
    result = json.load(check)
    
## appending them to a empty list
    
    people_results = []
    people_results.append(result)
    
## determining whether the list we created and the json file are equal
    
if process_people('/data/people.psv') == people_results:
    print('Both the lists are equal')
else:
    print('Both the lists are not equal')  
    

Both the lists are equal


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [10]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


[main 89039d8] Submitting the midterm
 2 files changed, 830 insertions(+), 2 deletions(-)
 create mode 100644 midterm/midterm-2022.ipynb
Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (6/6), 8.82 KiB | 4.41 MiB/s, done.
Total 6 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
remote: This repository moved. Please use the new location:
remote:   git@github.com:Sravani9544/hds5210-2022.git
To github.com:sravani9544/hds5210-2022.git
   332caff..89039d8  main -> main
